### Blockchain Capital Portfolio monthly momemtum strategy

In [57]:
# get category symbols
category = "blockchain-capital-portfolio"
import requests
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "category": category,
    "order": "market_cap_desc",
    "per_page": 100,
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

# Extract the symbols
defi_symbols = [coin['symbol'].upper() + '/USDT' for coin in data]
print("DeFi Symbols:", defi_symbols)

DeFi Symbols: ['XRP/USDT', 'UNI/USDT', 'ICP/USDT', 'STX/USDT', 'FIL/USDT', 'AAVE/USDT', 'TIA/USDT', 'WLD/USDT', 'THETA/USDT', 'AR/USDT', 'CKB/USDT', 'ROSE/USDT', 'ZK/USDT', '1INCH/USDT', 'ZRX/USDT', 'DCR/USDT', 'YFI/USDT', 'UMA/USDT', 'ACX/USDT', 'SUSHI/USDT', 'BAL/USDT', 'BB/USDT', 'POWR/USDT', 'CVC/USDT', 'COW/USDT', 'AUCTION/USDT', 'BADGER/USDT', 'EWT/USDT', 'WNXM/USDT', 'FLIP/USDT', 'FORT/USDT', 'OXT/USDT', 'FOX/USDT', 'PSP/USDT', 'INDEX/USDT', 'ARCH/USDT', 'WIT/USDT', 'HXRO/USDT', 'TNT/USDT', 'KINE/USDT', 'SAFE/USDT']


In [58]:
# Fetch aviable symbols from Bybit
import ccxt

exchange = ccxt.bybit()
exchange.load_markets()
all_symbols = exchange.symbols

# Filter to include only DeFi symbols available on the exchange
available_defi_symbols = [symbol for symbol in defi_symbols if symbol in all_symbols]
print("Available DeFi Symbols:", available_defi_symbols)

Available DeFi Symbols: ['XRP/USDT', 'UNI/USDT', 'ICP/USDT', 'STX/USDT', 'FIL/USDT', 'AAVE/USDT', 'TIA/USDT', 'WLD/USDT', 'THETA/USDT', 'AR/USDT', 'ROSE/USDT', 'ZK/USDT', '1INCH/USDT', 'ZRX/USDT', 'DCR/USDT', 'YFI/USDT', 'UMA/USDT', 'SUSHI/USDT', 'BB/USDT', 'FLIP/USDT', 'FORT/USDT', 'PSP/USDT', 'SAFE/USDT']


In [59]:
# Fetch ohlcv from Bybit
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import plotly.express as px
import plotly.graph_objects as go

symbols = available_defi_symbols
timeframe = '1d'
limit = 252

# Create an empty list to store DataFrames for each symbol
data_frames = []

for symbol in symbols:
    try:
        ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['symbol'] = symbol

        # Convert timestamp to datetime format for readability
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

        data_frames.append(df)
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")

# Concatenate all DataFrames into a single DataFrame
all_data = pd.concat(data_frames, ignore_index=True)
all_data.head()

,timestamp,open,high,low,close,volume,symbol
0,2024-03-10,0.6199,0.6277,0.5966,0.6081,1.214860e+08,XRP/USDT
1,2024-03-11,0.6081,0.7441,0.5812,0.7228,8.034530e+08,XRP/USDT
2,2024-03-12,0.7228,0.7329,0.6578,0.6880,4.895242e+08,XRP/USDT
3,2024-03-13,0.6880,0.7025,0.6684,0.6891,2.394229e+08,XRP/USDT
4,2024-03-14,0.6891,0.7075,0.6379,0.6691,4.721125e+08,XRP/USDT


In [60]:
# Check min max date
print(all_data['timestamp'].min(), all_data['timestamp'].max())

2024-03-10 00:00:00 2024-11-16 00:00:00


In [61]:
# Filter dublicates and unnecesary timestamps
# Ensure that 'timestamp' is in datetime format
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'])

# Find the latest date in the data
latest_date = all_data['timestamp'].max()

# Calculate the cutoff date for 252 days before the latest date
cutoff_date = latest_date - pd.Timedelta(days=252)

# Filter the data to include only rows within the last 252 days
filtered_data = all_data[all_data['timestamp'] >= cutoff_date]

# Check the new date range to confirm
print("Filtered data range:", filtered_data['timestamp'].min(), "-", filtered_data['timestamp'].max())

# Identify duplicate entries based on 'timestamp' and 'symbol'
duplicates = filtered_data[filtered_data.duplicated(subset=['timestamp', 'symbol'], keep=False)]
print("Duplicate entries:")
duplicates.info()

# Remove duplicate entries, keeping only the first occurrence
filtered_data = filtered_data.drop_duplicates(subset=['timestamp', 'symbol'], keep='first')


Filtered data range: 2024-03-10 00:00:00 - 2024-11-16 00:00:00
Duplicate entries:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  0 non-null      datetime64[ns]
 1   open       0 non-null      float64       
 2   high       0 non-null      float64       
 3   low        0 non-null      float64       
 4   close      0 non-null      float64       
 5   volume     0 non-null      float64       
 6   symbol     0 non-null      object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 0.0+ bytes


In [62]:
# get closed price data
df = filtered_data[['timestamp', 'close', 'symbol']]
df.head()

,timestamp,close,symbol
0,2024-03-10,0.6081,XRP/USDT
1,2024-03-11,0.7228,XRP/USDT
2,2024-03-12,0.6880,XRP/USDT
3,2024-03-13,0.6891,XRP/USDT
4,2024-03-14,0.6691,XRP/USDT


In [63]:
# pivot by timestamp and symbol
df_pivoted = df.pivot(index='timestamp', columns='symbol', values='close')

print(df_pivoted.head())
print(df_pivoted.tail())

symbol      1INCH/USDT  AAVE/USDT  AR/USDT  BB/USDT  DCR/USDT  FIL/USDT  \
timestamp                                                                 
2024-03-10      0.6425   126.8450  41.2125      NaN   27.3457    10.681   
2024-03-11      0.6616   136.5453  39.6196      NaN   28.9360    11.164   
2024-03-12      0.6582   135.3236  37.9058      NaN   30.3560    10.907   
2024-03-13      0.6837   142.6650  37.7186      NaN   31.0513    10.631   
2024-03-14      0.6567   139.4301  38.7300      NaN   29.7629    10.505   

symbol      FLIP/USDT  FORT/USDT  ICP/USDT  PSP/USDT  ...  SUSHI/USDT  \
timestamp                                             ...               
2024-03-10     8.8851     0.2509   14.1426  0.057288  ...       2.007   
2024-03-11     8.0133     0.3156   15.0341  0.059862  ...       2.077   
2024-03-12     7.9010     0.2892   14.7186  0.056332  ...       2.059   
2024-03-13     7.6185     0.2688   14.4990  0.056570  ...       2.067   
2024-03-14     7.4630     0.2635   1

In [64]:
# Calculate monthly returns
returns_df = df_pivoted.pct_change().resample("ME").agg(lambda x: (x+1).prod()-1)
# Convert the index of past_cum_return_df to timezone-aware UTC
returns_df.index = returns_df.index.tz_localize('UTC')
returns_df.tail(7)

symbol,1INCH/USDT,AAVE/USDT,AR/USDT,BB/USDT,DCR/USDT,FIL/USDT,FLIP/USDT,FORT/USDT,ICP/USDT,PSP/USDT,...,SUSHI/USDT,THETA/USDT,TIA/USDT,UMA/USDT,UNI/USDT,WLD/USDT,XRP/USDT,YFI/USDT,ZK/USDT,ZRX/USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-31 00:00:00+00:00,0.225856,0.244599,0.498428,0.348602,0.063583,0.019015,0.005342,0.039916,-0.079709,-0.294226,...,0.219307,0.057924,0.112506,0.332190,0.414731,0.064337,0.033546,0.044917,0.000000,0.123163
2024-06-30 00:00:00+00:00,-0.173077,-0.060403,-0.385295,-0.226649,-0.255435,-0.218696,-0.497666,-0.286465,-0.310355,-0.151511,...,-0.278830,-0.267852,-0.411932,-0.296722,-0.066272,-0.489087,-0.080178,-0.086524,-0.164074,-0.291458
2024-07-31 00:00:00+00:00,-0.148537,0.102874,0.045649,-0.172508,-0.188693,-0.060714,0.070565,-0.120612,0.095162,0.024389,...,-0.202864,-0.125152,-0.150371,0.027442,-0.228691,-0.147670,0.308339,-0.123103,-0.175973,-0.005973
2024-08-31 00:00:00+00:00,-0.297504,0.206741,-0.257002,-0.312835,-0.145906,-0.151616,-0.244390,-0.208628,-0.155476,-0.218522,...,-0.164671,-0.121301,-0.147958,-0.092740,-0.171003,-0.292019,-0.091026,-0.071080,-0.287474,-0.207594
2024-09-30 00:00:00+00:00,0.163043,0.208091,0.056939,0.273301,0.137567,0.068067,0.179322,0.001627,0.189378,0.025433,...,0.437276,0.166806,0.232226,0.161539,0.243344,0.243642,0.079830,0.027264,0.312200,0.153051
2024-10-31 00:00:00+00:00,-0.115025,-0.084801,-0.327150,-0.204141,-0.016376,-0.073171,-0.053219,-0.238830,-0.125936,-0.103527,...,-0.147132,-0.194113,-0.119698,-0.044002,0.030327,0.037445,-0.166503,-0.116040,-0.048316,-0.050523
2024-11-30 00:00:00+00:00,0.233550,0.207139,0.190507,0.182118,0.144545,0.248161,0.164617,0.011740,0.184647,0.358639,...,0.144737,0.304255,0.064413,-0.007631,0.191453,0.218900,1.049451,0.354340,0.133077,0.195529


In [65]:
# Create a line plot with Plotly
fig = px.line(returns_df, x=returns_df.index, y=returns_df.columns, title="Monthly returns")
# Adjust the layout for a wider plot
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Returns")
# Show the interactive plot
fig.show()

In [66]:
# rolling cumulative for 6 months return
past_cum_return_df = (returns_df + 1).rolling(6).apply(np.prod) - 1

past_cum_return_df.tail(7)

symbol,1INCH/USDT,AAVE/USDT,AR/USDT,BB/USDT,DCR/USDT,FIL/USDT,FLIP/USDT,FORT/USDT,ICP/USDT,PSP/USDT,...,SUSHI/USDT,THETA/USDT,TIA/USDT,UMA/USDT,UNI/USDT,WLD/USDT,XRP/USDT,YFI/USDT,ZK/USDT,ZRX/USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-31 00:00:00+00:00,-0.627704,0.022934,-0.475242,-0.406959,-0.609953,-0.665762,-0.879495,-0.510163,-0.465480,-0.739876,...,-0.721973,-0.628204,-0.720158,-0.521429,-0.582046,-0.843935,-0.068903,-0.478883,-0.509194,-0.637148
2024-09-30 00:00:00+00:00,-0.549547,0.220984,-0.410061,-0.244880,-0.574274,-0.617246,-0.766080,-0.468939,-0.521195,-0.663540,...,-0.575661,-0.545670,-0.613707,-0.402931,-0.428977,-0.774509,-0.028135,-0.416789,-0.355964,-0.560100
2024-10-31 00:00:00+00:00,-0.375919,0.720784,-0.491086,-0.399031,-0.385996,-0.371957,-0.543859,-0.606303,-0.389755,-0.559305,...,-0.282267,-0.440115,-0.486261,-0.030220,0.082037,-0.576634,0.017572,-0.293974,-0.387082,-0.313742
2024-11-30 00:00:00+00:00,-0.372002,0.668992,-0.595666,-0.473220,-0.339257,-0.230729,-0.471593,-0.616970,-0.214460,-0.151648,...,-0.326162,-0.309749,-0.508470,-0.277596,-0.088734,-0.515153,1.017774,-0.084904,-0.305516,-0.269527


In [67]:
# Show cumulative return from past 6 months
fig = px.line(past_cum_return_df, x=past_cum_return_df.index, y=past_cum_return_df.columns, title="Rolling cumulative for 6 months return")
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Cumulative Returns")
fig.show()

In [68]:
# Define formation and measurement period
# Define the dates with timezone information
end_of_measurement_period = dt.datetime(2024, 9, 30, tzinfo=pytz.UTC)
formation_period = dt.datetime(2024, 10, 31, tzinfo=pytz.UTC)

end_of_measurement_period_return_df = past_cum_return_df.loc[end_of_measurement_period]
# Transpose the DataFrame so that tickers become rows and returns become a single column
end_of_measurement_period_return_df = end_of_measurement_period_return_df.T
end_of_measurement_period_return_df = end_of_measurement_period_return_df.reset_index()
# Rename columns for clarity
end_of_measurement_period_return_df.columns = ['Symbol', 'Return']
end_of_measurement_period_return_df.head()

,Symbol,Return
0,1INCH/USDT,-0.549547
1,AAVE/USDT,0.220984
2,AR/USDT,-0.410061
3,BB/USDT,-0.244880
4,DCR/USDT,-0.574274


In [69]:
# highest momentum in the positive direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmax()]

Symbol    AAVE/USDT
Return     0.220984
Name: 1, dtype: object

In [70]:
# highest momentum in the negative direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmin()]

Symbol    WLD/USDT
Return   -0.774509
Name: 18, dtype: object

In [71]:
# rank symbols with quantiles
end_of_measurement_period_return_df['rank'] = pd.qcut(end_of_measurement_period_return_df.iloc[:,1], 11, labels=False)
end_of_measurement_period_return_df.head(7)

,Symbol,Return,rank
0,1INCH/USDT,-0.549547,4
1,AAVE/USDT,0.220984,10
2,AR/USDT,-0.410061,8
3,BB/USDT,-0.244880,9
4,DCR/USDT,-0.574274,3
5,FIL/USDT,-0.617246,1
6,FLIP/USDT,-0.766080,0


In [72]:
# Create a bar plot to visualize ranks
fig = px.bar(
    end_of_measurement_period_return_df, 
    x='Symbol', 
    y='Return', 
    color='rank',  # Color by rank to visually distinguish groups
    title="Asset Returns and Ranks",
    labels={'rank': 'Rank'},
)

# Customize layout for readability
fig.update_layout(
    xaxis_title="Symbol",
    yaxis_title="Return",
    coloraxis_colorbar=dict(title="Rank"),
    width=1000,
    height=600
)

fig.show()


In [73]:
# select highest rank
long_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==10,"Symbol"].values
long_stocks

array(['AAVE/USDT', 'XRP/USDT'], dtype=object)

In [74]:
# select lowest rank
short_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==0,"Symbol"].values
short_stocks

array(['FLIP/USDT', 'PSP/USDT', 'WLD/USDT'], dtype=object)

In [75]:
# long strategy return
from dateutil.relativedelta import relativedelta

long_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(long_stocks)]
long_return_df

symbol
AAVE/USDT    0.207139
XRP/USDT     1.049451
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [76]:
# short strategy return
short_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(short_stocks)]
short_return_df

symbol
FLIP/USDT    0.164617
PSP/USDT     0.358639
WLD/USDT     0.218900
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [78]:
# Define Strategy tester class
class strategyTester:
    """Calculate metrics based on cumulative return"""

    def __init__(self, df: pd.Series, period: str, riskfree_rate: float = 0.03, initial_wealth: int = 1000):
        self.annualized_return_value = self.annualized_return(df, period)
        self.annualized_volatility_value = self.annualized_volatility(df, period)
        self.sharpe_ratio_value = self.sharpe_ratio(df, riskfree_rate)
        self.max_drawdown_value = self.max_drawdown(df, initial_wealth)
        
    # annualized return
    @staticmethod
    def annualized_return(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        annualized_return = (1 + df) ** (factor / len(df)) - 1
        print(f"Annualized return: {annualized_return}")
        return annualized_return

    # annualized volatility
    @staticmethod
    def annualized_volatility(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        volatility = df.std() * (factor ** 0.5)
        print(f"Annualized volatility: {volatility}")
        return volatility

    # Sharpe ratio
    @staticmethod
    def sharpe_ratio(df: pd.Series, riskfree_rate: float = 0.03):
        excess_return = df.mean() - riskfree_rate
        volatility = df.std()
        sharpe_ratio = excess_return / volatility if volatility != 0 else np.nan
        print(f"Sharpe ratio: {sharpe_ratio}")
        return sharpe_ratio

    # Drawdown calculation
    @staticmethod
    def drawdown(return_series: pd.Series, initial_wealth: float = 100):
        prior_peaks_series = return_series.cummax()
        drawdown_series = (return_series - prior_peaks_series) / prior_peaks_series
        return pd.DataFrame({
            "Wealth Index": return_series,
            "Prior Peaks": prior_peaks_series,
            "Drawdown": drawdown_series
        })
    
    # Max drawdown
    @staticmethod
    def max_drawdown(df: pd.Series, initial_wealth):
        drawdown_df = strategyTester.drawdown(df, initial_wealth)
        max_drawdown = drawdown_df["Drawdown"].min()
        print(f"Max drawdown: {max_drawdown} with initial wealth: {initial_wealth}")
        return max_drawdown


In [79]:
# long momentum strategy test
long_momentum_strategy_test = strategyTester(long_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
AAVE/USDT     2.094163
XRP/USDT     73.101101
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 2.063234077546209
Sharpe ratio: 1.0045168941574942
Max drawdown: 0.0 with initial wealth: 1000


In [80]:
# short momentum strategy test
short_momentum_strategy_test = strategyTester(short_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
FLIP/USDT    0.839641
PSP/USDT     2.407350
WLD/USDT     1.207352
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 0.34675151190251946
Sharpe ratio: 2.1717138508676643
Max drawdown: -0.38963880613585555 with initial wealth: 1000


In [77]:
# Comparison of monthly Long and Short momentum strategies
# Initialize an empty figure
fig = go.Figure()

# Add the first strategy (e.g., long strategy)
fig.add_trace(go.Bar(x=long_return_df.index, y=long_return_df.values, marker_color='blue', name="Long"))

# Add the second strategy (e.g., short strategy)
fig.add_trace(go.Bar(x=short_return_df.index, y=short_return_df.values, marker_color='red', name="Short"))

# Update layout for title and axis labels
fig.update_layout(
    title="Comparison of monthly Long and Short momentum strategies",
    width=1400,
    height=500,
    xaxis_title="Symbol",
    yaxis_title="Cumulative Returns",
    legend_title="Strategies",
)

# Show the plot
fig.show()

In [81]:
# Annualized Return and Max Drawdown from Long and Short Strategies
# Initialize an empty figure
fig = go.Figure()

# Add traces for Long strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Long)", "Max Drawdown (Long)"], 
    y=[long_momentum_strategy_test.annualized_return_value.mean(), long_momentum_strategy_test.max_drawdown_value], 
    marker_color='blue', 
    name="Long"
))

# Add traces for Short strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Short)", "Max Drawdown (Short)"], 
    y=[short_momentum_strategy_test.annualized_return_value.mean(), short_momentum_strategy_test.max_drawdown_value], 
    marker_color='red', 
    name="Short"
))

# Update layout
fig.update_layout(
    title="Annualized Return and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Values",
    barmode='group',  # Group bars by category
    width=800,
    height=400
)

# Show the plot
fig.show()



In [82]:
# Annualized Volatility and Max Drawdown from Long and Short Strategies
# Create a figure
fig = go.Figure()

# Add bars for the Long Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(long)", "Sharpe Ratio(long)"], 
    y=[long_momentum_strategy_test.annualized_volatility_value, long_momentum_strategy_test.sharpe_ratio_value],
    name="Long",
    marker_color='blue'
))

# Add bars for the Short Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(short)", "Sharpe Ratio(short)"], 
    y=[short_momentum_strategy_test.annualized_volatility_value, short_momentum_strategy_test.sharpe_ratio_value],
    name="Short",
    marker_color='red'
))

# Update layout for grouped bars
fig.update_layout(
    title="Annualized Volatility and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Metric Value",
    barmode='group',  # Group bars side-by-side
    width=800,
    height=400
)

# Show the plot
fig.show()
